# DS Challenge

## Exercise 1

In [1]:
import bz2

def count_lines(file_path):
    count = 0
    with bz2.BZ2File(file_path, 'r') as file:
        for line in file:
            count += 1
    return count

In [2]:
%%time
count_lines('data/bookings.csv.bz2')

CPU times: user 2min 35s, sys: 426 ms, total: 2min 35s
Wall time: 2min 36s


10000011

In [3]:
%%time
count_lines('data/searches.csv.bz2')

CPU times: user 2min 27s, sys: 356 ms, total: 2min 27s
Wall time: 2min 27s


20390199

## Exercise 2

In [4]:
import pandas as pd
chunksize = 1000
bookings = pd.read_csv('data/bookings.csv.bz2', sep='^', chunksize=chunksize)

In [5]:
df = bookings.read(1000)

In [6]:
df.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,...,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


In [7]:
pax_by_airport = df.groupby('arr_port')['pax'].sum()

In [8]:
pax_by_airport.sort_values(ascending=False).head(10)

arr_port
DCA         50
JFK         14
FLL         13
PVG         13
SVO         12
LGA         12
CGK         12
MIA         11
LAS         11
BOS         11
Name: pax, dtype: int64

In [9]:
def top10_arr_airports_2013(bookings_filepath, chunksize=1e7):
    bookings = pd.read_csv(bookings_filepath, sep='^', usecols=['pax','arr_port', 'year'], chunksize=chunksize)
    total_pax_by_airport = pd.Series([],dtype=int)
    for chunk in bookings:
        year_2013 = chunk[chunk['year'] == 2013]
        pax_by_airport = year_2013.groupby('arr_port')['pax'].sum()
        total_pax_by_airport = total_pax_by_airport.add(pax_by_airport, fill_value=0)
    return total_pax_by_airport.sort_values(ascending=False).head(10)

In [10]:
%%time
top10_arr_airports_2013('data/bookings.csv.bz2')

CPU times: user 2min 43s, sys: 743 ms, total: 2min 44s
Wall time: 2min 44s


arr_port
LHR         88809.0
MCO         70930.0
LAX         70530.0
LAS         69630.0
JFK         66270.0
CDG         64490.0
BKK         59460.0
MIA         58150.0
SFO         58000.0
DXB         55590.0
dtype: float64

## Exercise 3